# PLAN

- [ ] Acquisition
    - [x] Select what list of repos to scrape.
    - [x] Get requests form the site.
    - [ ] Save responses to csv.
- [ ] Preparation
    - [ ] Prepare the data for analysis.
- [ ] Exploration
    - [ ] Answer the following prompts:
        - [ ] What are the most common words in READMEs?
        - [ ] What does the distribution of IDFs look like for the most common words?
        - [ ] Does the length of the README vary by language?
        - [ ] Do different languages use a different number of unique words?
- [ ] Modeling
    - [ ] Transform the data for machine learning; use language to predict.
    - [ ] Fit several models using different text repressentations.
    - [ ] Build a function that will take in the text of a README file, and makes a prediction of language.
- [ ] Delivery
    - [ ] Github repo
        - [x] This notebook.
        - [ ] Documentation within the notebook.
        - [ ] README file in the repo.
        - [ ] Python scripts if applicable.
    - [ ] Google Slides
        - [ ] 1-2 slides only summarizing analysis.
        - [ ] Visualizations are labeled.
        - [ ] Geared for the general audience.
        - [ ] Share link @ readme file and/or classroom.

# ENVIRONMENT

In [135]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")

import unicodedata
import re
import os
from requests import get
import json
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import csv
import time
import pandas as pd

BASEURL = 'https://github.com/search?p=1&q=stars%3A%3E0&s=stars&type=Repositories'
HEADERS = {'User-Agent': 'Maybe Sentient Attack Helicoptor'}

# ACQUIRE

First thing that needs to happen is to get the links from the most starred github repositories.  The most complicated part here was identifying the section that had the actual urls.

The flow goes something like this:
* get a response from the BASEURL
* set number of pages to scrape and loop through all of them
* find the list of all the repos on the page
* from that list find the individual list item repos
* do a check to see if there is a language associated with the repo
* * if no language, skip
* loop through individual repo sections and grab the url
* print out a list of the total valid urls scraped
* save the resulting list of urls to a csv
* return the urls

In [117]:
def get_url_list(page):
    urls = []
    response = get(BASEURL, headers=HEADERS)
    soup = BeautifulSoup(response.content)
    max_page = page + 1
    for i in range(1,max_page):
        url = 'https://github.com/search?p=' + str(i) + '&q=stars%3A%3E0&s=stars&type=Repositories'
#         print(f'traversing url: {url}')
        response = get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text)
        list_of_repos = soup.find('ul', class_='repo-list')
        repository = list_of_repos.find_all('li', class_='repo-list-item')
        for h in repository:
            if h.find(attrs={'itemprop':'programmingLanguage'}):
                a = h.find('a')
                urls.append(a.attrs['href'])
        time.sleep(3)
    print(f'Scraped a total of {len(urls)} github urls.')
    urls = ['https://github.com' + url for url in urls]
    with open('github_urls.csv', 'w') as f:
        ghub_urls = csv.writer(f, delimiter=',')
        ghub_urls.writerow(urls)
    return urls


In [118]:
get_url_list(25)

traversing url: https://github.com/search?p=1&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=2&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=3&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=4&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=5&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=6&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=7&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=8&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=9&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=10&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search?p=11&q=stars%3A%3E0&s=stars&type=Repositories
traversing url: https://github.com/search

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/996icu/996.ICU',
 'https://github.com/vuejs/vue',
 'https://github.com/twbs/bootstrap',
 'https://github.com/facebook/react',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/airbnb/javascript',
 'https://github.com/electron/electron',
 'https://github.com/facebook/create-react-app',
 'https://github.com/vinta/awesome-python',
 'https://github.com/flutter/flutter',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/nodejs/node',
 'https://github.com/FortAwesome/Font-Awesome',
 'https://github.com/angular/angular.js',
 'https://github.com/golang/go',
 'https://github.com/vinta/awesome-python',
 'https://github.com/flutter/flutter',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/nodejs/node',
 'https://github.com/FortAwesome/Font-Awesome',
 'https://github.com/angular/

In [120]:
def grab_readmes_and_languages(urls):
    readmes = []
    languages = []
    for url in urls:
        response = get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        print('Retrieving README')
        if soup.find('div', class_='Box-body') == None:
            print('Skipping because of no README')
            continue
        else:
            single_readme = soup.find('div', class_='Box-body').text
            print('Got README')
        print('Retrieving language')
        if soup.find('span', class_='lang') == None:
            print('Skipping because of no language')
            continue
        else:
            repo_language = soup.find('span', class_='lang').text
            print('Got language')
        languages.append(repo_language)
        readmes.append(single_readme)
    df = pd.DataFrame({'readme':readmes, 'language':languages})
     
    return df


In [144]:
with open('github_urls.csv') as f:
    urls = f.readlines()
urls = urls[0].split(',')

In [146]:
df = grab_readmes_and_languages(urls)

Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrievi

Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrieving README
Got README
Retrieving language
Got language
Retrievi

In [137]:
urls

['https://github.com/freeCodeCamp/freeCodeCamp,https://github.com/996icu/996.ICU,https://github.com/vuejs/vue,https://github.com/twbs/bootstrap,https://github.com/facebook/react,https://github.com/tensorflow/tensorflow,https://github.com/airbnb/javascript,https://github.com/electron/electron,https://github.com/facebook/create-react-app,https://github.com/vinta/awesome-python,https://github.com/flutter/flutter,https://github.com/donnemartin/system-design-primer,https://github.com/CyC2018/CS-Notes,https://github.com/nodejs/node,https://github.com/FortAwesome/Font-Awesome,https://github.com/angular/angular.js,https://github.com/golang/go,https://github.com/vinta/awesome-python,https://github.com/flutter/flutter,https://github.com/donnemartin/system-design-primer,https://github.com/CyC2018/CS-Notes,https://github.com/nodejs/node,https://github.com/FortAwesome/Font-Awesome,https://github.com/angular/angular.js,https://github.com/daneden/animate.css,https://github.com/axios/axios,https://git

# PREPARE

# EXPLORE

# MODEL